## Create Sampled Dataset of Waymo


In [25]:
import argparse
import os
import yaml
import numpy as np
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import concurrent.futures
import random

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

import os
import time
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
import numpy as np

tf.enable_eager_execution()

from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset
from waymo_open_dataset.protos import segmentation_metrics_pb2
from waymo_open_dataset.protos import segmentation_submission_pb2

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
dataroot = '/media/scratch/berlukas/waymo/segments'
segments = os.listdir(dataroot)

print(f'Setting dataroot to {dataroot}.')
print(f'Found {len(segments)} elements in the folder.')

Setting dataroot to /media/scratch/berlukas/waymo/segments.
Found 0 elements in the folder.


In [3]:
def convert_range_image_to_point_cloud_labels(frame,
                                              range_images,
                                              segmentation_labels,
                                              ri_index=0):
    """Convert segmentation labels from range images to point clouds.

    Args:
    frame: open dataset frame
    range_images: A dict of {laser_name, [range_image_first_return,
       range_image_second_return]}.
    segmentation_labels: A dict of {laser_name, [range_image_first_return,
       range_image_second_return]}.
    ri_index: 0 for the first return, 1 for the second return.

    Returns:
    point_labels: {[N, 2]} list of 3d lidar points's segmentation labels. 0 for
      points that are not labeled.
    """
    calibrations = sorted(frame.context.laser_calibrations, key=lambda c: c.name)
    point_labels = []
    for c in calibrations:
        range_image = range_images[c.name][ri_index]
        range_image_tensor = tf.reshape(
            tf.convert_to_tensor(range_image.data), range_image.shape.dims)
        range_image_mask = range_image_tensor[..., 0] > 0

        if c.name in segmentation_labels:
            sl = segmentation_labels[c.name][ri_index]
            sl_tensor = tf.reshape(tf.convert_to_tensor(sl.data), sl.shape.dims)
            sl_points_tensor = tf.gather_nd(sl_tensor, tf.where(range_image_mask))
        else:
            num_valid_point = tf.math.reduce_sum(tf.cast(range_image_mask, tf.int32))
            sl_points_tensor = tf.zeros([num_valid_point, 2], dtype=tf.int32)

        point_labels.append(sl_points_tensor.numpy())
    return point_labels

In [5]:
all_sem_clouds = []
k = 0    
for segment in tqdm(segments):
    FILENAME = f'{dataroot}/{segment}'    
    dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))

        if frame.lasers[0].ri_return1.segmentation_label_compressed:
            (range_images, camera_projections, segmentation_labels, range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)
            points, cp_points = frame_utils.convert_range_image_to_point_cloud(frame, range_images, camera_projections, range_image_top_pose, keep_polar_features=True)
            point_labels = convert_range_image_to_point_cloud_labels(frame, range_images, segmentation_labels)

            points_all = np.concatenate(points, axis=0) # 3d points in vehicle frame.
            points_all = points_all[:, [3, 4, 5, 1]]
            point_labels_all = np.concatenate(point_labels, axis=0) # point labels.
            point_labels_all = np.reshape(point_labels_all[:,1], (point_labels_all.shape[0], 1))

            point_xyzil = np.hstack((points_all, point_labels_all))
            all_sem_clouds.append(point_xyzil)

print(f'Finished Loading {len(all_sem_clouds)} clouds.')

  0%|          | 0/25 [00:00<?, ?it/s]

Finished Loading 710 clouds.


In [6]:
filename = 'archive11'
np.save(f'{dataroot}/{filename}.npy', all_sem_clouds)
print(f'Saved clouds to {dataroot}/{filename}.npy')

/home/berlukas/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Saved clouds to /media/scratch/berlukas/waymo/segments/archive11.npy


In [32]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):    
    sample = all_sem_clouds[sample_idx]
    sample_sphere = Sphere(sample)
    # sample_sphere = all_sem_clouds[sample_idx]
    features = sample_sphere.sampleUsingGrid(grid)
    return features

print(f"Computing features for {len(all_sem_clouds)} clouds.")
bw = 50
grid, _ = DHGrid.CreateGrid(bw)

sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
start_time = time.time() * 1000
sem_features = process_map(sample_func, sem_idx, max_workers=16)
executionTime = (time.time() * 1000 - start_time)

print(f"It took {executionTime / len(all_sem_clouds)} ms on average per sample.")

Computing features for 500 clouds.


  0%|          | 0/500 [00:00<?, ?it/s]

It took 21.8641162109375 ms on average per sample.


In [33]:
filename = f"{dataroot}/../extracted/features_50/features02.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

Wrote features to /media/scratch/berlukas/waymo/segments/../extracted/features_50/features02.npy.


# Resample Archive

In [30]:
archive = 'archive02'
all_sem_clouds = np.load(f'{dataroot}/../extracted/pointclouds/{archive}.npy', allow_pickle=True)
print(f'Loaded {len(all_sem_clouds)} from {archive}.')

Loaded 817 from archive02.


In [31]:
np.random.shuffle(all_sem_clouds)
all_sem_clouds = all_sem_clouds[0:500]